In [6]:
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import re
# from scipy.spatial import distance_matrix

In [7]:
from util import load_data, generate_statistics

In [8]:
stopwords = ['', "wasn't", 'there', 'been', 'they', 'until', 'yourself', 'that', 'themselves', 'into', 'where', 'he', 'both', "it's", 'too', 'under', 'no', 'in', 'the', 'me', 'which', 'do', 'i', 'own', 'their', 'him', "that'll", 'up', '&amp;', 'same', 'at', 'a', 'rt', "aren't", 'who', 'on', 'we', 'being', 'can', 'against', 'your', 'she', 'again', 'whom', 'will', 'as', 'did', 'has', "isn't", 'few', 'through', 'down', "should've", "you'll", 'myself', 'himself', 'such', 'am', 'tip', 'most', 'but', 'while', 'to', 'if', "weren't", "won't", 'us', 'and', 'how', 'be', 'each', 'some', 'may', 'says', 'about', 'for', 'any', 'only', 'are', 'his', 'with', 'by', 'here', "doesn't", 'all', 'then', 'of', 'hers', 'them', 'or', 'theirs', 'because', 'having', 'itself', 'what', 'yourselves', "shouldn't", 'its', 'were', 'during', "didn't", 'further', 'nor', 'out', 'had', "you'd", 'not', 'you', 'could', 'our', 'my', 'other', 'herself', 'after', 'her', 'those', 'why', 'have', "hasn't", "you're", 'once', 'doing', 'below', 'yours', "she's", '--', 'is', 'off', 'an', 'via', "hadn't", 'from', 'ours', 'so', "wouldn't", "you've", 'this', 'over', 'it', 'between', 'when', "haven't", 'more', 'before', "couldn't", 'than', 'very', 'these', 'ourselves', "don't", 'above', 'does', 'was', 'should', 'rt', 'would', 'w/', 'just']

In [9]:
X,vocab = load_data('data/cnnhealth.txt', stopwords)

Importing data...


/opt/anaconda3/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3213: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).shape[axis]


X has 4061 examples and 13552 features in BOW format...


In [5]:
def distance1(x1,x2):
    """
    nonlocal, normalized edit distance
    """
    return np.sum(np.abs(x1-x2))/(np.sum(x1)+np.sum(x2))

def distance2(x1,x2):
    """
    Euclidean distance
    """
    return np.sqrt(np.sum(np.power((x1-x2), 2)))

In [12]:
def create_distance_matrix(X:np.ndarray, distance_type=1) -> np.ndarray:
    assert distance_type in (1,2)
    '''
    return distance_matrix(X,X, p=distance_type, threshold=1000000)
    if distance_type == 1:
        return np.linalg.norm(X[:, None, :] - X[None, :, :], axis=-1, ord=1)
    else:
        return np.linalg.norm(X[:, None, :] - X[None, :, :], axis=-1, ord=2)
    '''
    if distance_type == 1:
        distance_measure = distance1
    else:
        distance_measure = distance2
    m,n = np.shape(X)
    distance_matrix = np.zeros(shape=(m,m))
    for i in range(m):
        print(str(i/m), end='\r')
        for j in range(i):
            distance_matrix[i,j] = distance_measure(X[i],X[j])
            distance_matrix[j,i] = distance_matrix[i,j]
    return distance_matrix

In [13]:
distance_matrix_1 = create_distance_matrix(X,distance_type=1)

In [14]:
distance_matrix_2 = create_distance_matrix(X,distance_type=2)

In [15]:
'''
with open('distance_matrices.npy', 'wb') as file:
    np.save(file, distance_matrix_1)
    np.save(file, distance_matrix_2)
'''

In [6]:
with open('distance_matrices.npy', 'rb') as file:
    distance_matrix_1 = np.load(file)
    distance_matrix_2 = np.load(file)

In [10]:
generate_statistics(X, vocab)

Dataset Statistics:
# Number of Tweets: 4061
# Number of Words (Total): 36504
# Number of Tokens: 13552
# Average Number of Words per Tweet: 8.989
# Top 5 Legal Tokens: ['#getfit' 'new' '@cnnhealth' "today's" 'health' 'know' '@drsanjaygupta'
 'kids' '#health' 'one' 'help' '5' 'brain' 'cancer' 'study' '@cnn' 'story'
 'see' '#ebola' 'make' 'food' 'life' 'healthy' 'time' 'say' 'day' 'like'
 'need' 'good' "here's" 'just' 'want' 'lost' 'join' '10' '@cnnfitnation'
 'pounds' 'now' 'check' 'love' 'watch' 'ways' 'risk' 'medical'
 '#weightloss' 'heart' 'use' 'people' 'first' 'things']


In [9]:
ntlk = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [10]:
def diff(x:list,y:list):
    res = []
    for item in x:
        if item not in y:
            res.append(item)
    return res

In [20]:
# items in my list that aren't in ntlk
diff(stopwords, ntlk)

['may',
 'thatdous',
 '--',
 'rt',
 'us',
 '&amp;',
 'tip',
 'via',
 'says',
 'get',
 'could',
 '']

In [21]:
# items in ntlk that aren't in my list
print(diff(ntlk, stopwords))

['s', 't', 'just', 'don', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']


In [ ]:
just 

In [11]:
re.sub('#','','people with #ebola are sos ick')

'people with ebola are sos ick'

In [ ]:
re.sub('#','','people with #ebola are sos ick')